In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [63]:
%matplotlib inline
plt.style.use('ggplot')

In [64]:
houses = pd.read_csv('../test.csv')

In [69]:
MSSubClass = \
'''
20:1-STORY 1946 & NEWER ALL STYLES
30:1-STORY 1945 & OLDER
40:1-STORY W/FINISHED ATTIC ALL AGES
45:1-1/2 STORY - UNFINISHED ALL AGES
50:1-1/2 STORY FINISHED ALL AGES
60:2-STORY 1946 & NEWER
70:2-STORY 1945 & OLDER
75:2-1/2 STORY ALL AGES
80:SPLIT OR MULTI-LEVEL
85:SPLIT FOYER
90:DUPLEX - ALL STYLES AND AGES
120:1-STORY PUD (Planned Unit Development) - 1946 & NEWER
150:1-1/2 STORY PUD - ALL AGES
160:2-STORY PUD - 1946 & NEWER
180:PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
190:2 FAMILY CONVERSION - ALL STYLES AND AGES
'''

MSSubClass_split = MSSubClass.strip().split('\n')
MSSubClass_dict = {int(k.split(':')[0]): k.split(':')[1] for k in MSSubClass_split}

In [70]:
houses['MSSubClass'] = houses['MSSubClass'].apply(lambda x: MSSubClass_dict[x])

In [73]:
dtypes = pd.Series([houses[x].dtype for x in houses.columns])

In [74]:
is_numeric = dtypes.apply(lambda x: True if x == int or x == float else False)

In [78]:
num_cols = houses.columns[is_numeric]

In [79]:
cat_cols = houses.columns[~is_numeric]

In [85]:
hdummy = houses.copy()

for col in cat_cols:
    for value in houses[col].unique():
        hdummy['{}${}'.format(col,value)] = hdummy[col] == value
    hdummy.drop(col, axis=1, inplace=True)

In [87]:
hdummy.to_csv('../data/test_withdummy.csv')